In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.2'
spark_version = 'spark-3.2.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Fetched 261 kB in 3s (76.3 kB/s)
Reading package l

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-10-14 21:21:46--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.2’

postgresql-42.2.16. 100%[===================>] 979.38K  5.88MB/s    in 0.2s    

2022-10-14 21:21:46 (5.88 MB/s) - ‘postgresql-42.2.16.jar.2’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Video_Games_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...| 2015-08-31|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...|     Video Games|          5|    

### Create DataFrames to match tables

In [5]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame


In [6]:
# Create the vine_table DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RTIS3L2M1F5SM|          5|            0|          0|   N|                Y|
| R1ZV7R40OLHKD|          5|            0|          0|   N|                Y|
|R3BH071QLH8QMC|          1|            0|          1|   N|                Y|
|R127K9NTSXA2YH|          3|            0|          0|   N|                Y|
|R32ZWUXDJPW27Q|          4|            0|          0|   N|                Y|
|R3AQQ4YUKJWBA6|          1|            0|          0|   N|                Y|
|R2F0POU5K6F73F|          5|            0|          0|   N|                Y|
|R3VNR804HYSMR6|          5|            0|          0|   N|                Y|
| R3GZTM72WA2QH|          5|            0|          0|   N|                Y|
| RNQOY62705W1K|          4|            0|          0|   N|     

In [7]:
# Create new DataFrame to retrive all rows where total_votes >20
filter_df = vine_df.filter("total_votes >= 20")
filter_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R4PKAZRQJJX14|          1|           21|         34|   N|                N|
|R2CI0Y288CC7E2|          1|           21|         35|   N|                Y|
|R127WEQY2FM1T3|          1|          147|        175|   N|                Y|
|R3EZ0EPYLDA34S|          1|           14|         31|   N|                Y|
|R2FJ94555FZH32|          2|           55|         60|   N|                N|
|R1U3AR67RE273L|          1|           51|         65|   N|                Y|
|R3PZOXA5X1U8KW|          4|           31|         36|   N|                N|
| R6KTC1OPIOIIG|          2|           19|         34|   N|                Y|
|R36O341WWXXKNP|          5|           28|         31|   N|                N|
|R3GSK9MM8DNOYI|          1|            4|         32|   N|     

In [8]:
# Filter the Dataframe and create new Dataframe to retrive all rows where helpful_votes devided by total_votes >=50%
helpful_votes_df = filter_df.filter((filter_df["helpful_votes"] / filter_df["total_votes"]) >= .5)
helpful_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R4PKAZRQJJX14|          1|           21|         34|   N|                N|
|R2CI0Y288CC7E2|          1|           21|         35|   N|                Y|
|R127WEQY2FM1T3|          1|          147|        175|   N|                Y|
|R2FJ94555FZH32|          2|           55|         60|   N|                N|
|R1U3AR67RE273L|          1|           51|         65|   N|                Y|
|R3PZOXA5X1U8KW|          4|           31|         36|   N|                N|
| R6KTC1OPIOIIG|          2|           19|         34|   N|                Y|
|R36O341WWXXKNP|          5|           28|         31|   N|                N|
|R10LZVBLQHBVJ0|          2|          151|        198|   N|                N|
|R1VR5GLGY1GE7N|          1|           49|         51|   N|     

In [9]:
# Create a new Dataframe that retrieves all rows where vine==Y
vine_reviews_Y_df = helpful_votes_df.filter(helpful_votes_df["vine"] == 'Y')
vine_reviews_Y_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3KKUSGFZWSUIY|          5|           56|         63|   Y|                N|
|R10FO5UKKVZBK2|          3|           23|         23|   Y|                N|
| RM4KSGEOR7MU1|          5|           19|         24|   Y|                N|
| RG7VRMYLEXD23|          4|           22|         26|   Y|                N|
|R11O4YSCPSNL6L|          3|           20|         26|   Y|                N|
|R286MFBAJ8NPD6|          5|           46|         51|   Y|                N|
|R1JRR530H4COA2|          5|           22|         28|   Y|                N|
| RQ5WD90PUNBU9|          5|           21|         24|   Y|                N|
|R12648VHCQWUV9|          4|           21|         28|   Y|                N|
|R3KAW29CJ8L6DQ|          5|           17|         20|   Y|     

In [10]:
# Create a new Dataframe that retrieves all rows where vine==N
vine_reviews_N_df = helpful_votes_df.filter(helpful_votes_df["vine"] == 'N')
vine_reviews_N_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R4PKAZRQJJX14|          1|           21|         34|   N|                N|
|R2CI0Y288CC7E2|          1|           21|         35|   N|                Y|
|R127WEQY2FM1T3|          1|          147|        175|   N|                Y|
|R2FJ94555FZH32|          2|           55|         60|   N|                N|
|R1U3AR67RE273L|          1|           51|         65|   N|                Y|
|R3PZOXA5X1U8KW|          4|           31|         36|   N|                N|
| R6KTC1OPIOIIG|          2|           19|         34|   N|                Y|
|R36O341WWXXKNP|          5|           28|         31|   N|                N|
|R10LZVBLQHBVJ0|          2|          151|        198|   N|                N|
|R1VR5GLGY1GE7N|          1|           49|         51|   N|     

Paid_reviews

In [11]:
# Determine the total number of reviews
paid_reviews = vine_reviews_Y_df.count()
paid_reviews

94

In [12]:
# Determine the total number of 5 star reviews
paid_five_star_reviews = vine_reviews_Y_df.filter(vine_reviews_Y_df["star_rating"] == '5').count()
paid_five_star_reviews

48

In [13]:
# Determine the percentage of 5-star reviews for the two types of review (paid)
pct_paid_reviews = paid_five_star_reviews / paid_reviews
pct_paid_reviews

0.5106382978723404

Unpaid_reviews

In [14]:
# Determine the total number of reviews
unpaid_reviews = vine_reviews_N_df.count()
unpaid_reviews

40471

In [15]:
# Determine the total number of 5 star reviews
unpaid_five_star_reviews = vine_reviews_N_df.filter(vine_reviews_N_df["star_rating"] == '5').count()
unpaid_five_star_reviews

15663

In [16]:
# Determine the percentage of 5-star reviews for the two types of review (unpaid)
pct_unpaid_reviews = unpaid_five_star_reviews / unpaid_reviews
pct_unpaid_reviews

0.38701786464381904

### Find out the total number of 4 star reviews among Vine and non-Vine members

In [17]:
# Determine the total number of 4 star reviews Paid
paid_four_star_reviews = vine_reviews_Y_df.filter(vine_reviews_Y_df["star_rating"] == '4').count()
paid_four_star_reviews

24

In [18]:
# Determine the total number of 4 star reviews Unpaid
unpaid_four_star_reviews = vine_reviews_N_df.filter(vine_reviews_N_df["star_rating"] == '4').count()
unpaid_four_star_reviews

6738

In [19]:
# Determine the percentage of 4-star reviews for the two types of review (paid)
pct_paid_reviews = paid_four_star_reviews / paid_reviews
pct_paid_reviews

0.2553191489361702

In [20]:
# Determine the percentage of 4-star reviews for the two types of review (unpaid)
pct_unpaid_reviews = unpaid_four_star_reviews / unpaid_reviews
pct_unpaid_reviews

0.16648958513503495

In [52]:
# Determine the total number of 3 star reviews Unpaid
unpaid_three_star_reviews = vine_reviews_N_df.filter(vine_reviews_N_df["star_rating"] == '3').count()
unpaid_three_star_reviews

95